In [1]:
import pandas as pd
import numpy as np
import datetime
import time
import json
import random
import cvxpy as cvx
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
from operator import itemgetter

plotly.tools.set_credentials_file(username='Shenghao1993', api_key='EsC2v5QMJhXCNQHwX5gY')

# Acquire ETF price data

In [2]:
etf_prices = pd.read_csv('data/prices.txt', sep=',')
etf_prices = etf_prices.rename(columns={etf_prices.columns[0]: "date"}).set_index('date')
etf_prices = etf_prices.iloc[:, :-1]
etf_prices.head()

,AGG,DBC,DFE,DIA,DXJ,EEM,EFA,EWG,EWH,EWI,...,XLE,XLF,XLI,XLK,XLP,XLU,XLV,XLY,XME,XOP
date,,,,,,,,,,,,,,,,,,,,,
2007-01-02,99.91,23.80,65.66,124.47,55.28,38.34,73.51,27.15,16.16,33.42,...,56.58,36.92,35.23,23.33,26.29,36.82,33.50,38.52,47.66,36.64
2007-01-03,99.91,23.80,65.66,124.47,55.28,38.34,73.51,27.15,16.16,33.42,...,56.58,36.92,35.23,23.33,26.29,36.82,33.50,38.52,47.66,36.64
2007-01-04,100.12,23.18,65.60,124.68,55.95,37.81,73.32,27.06,15.99,33.34,...,55.55,36.80,35.08,23.69,26.33,36.77,33.88,38.51,47.42,36.16
2007-01-05,100.05,23.20,65.02,123.74,54.78,36.71,72.20,26.61,15.87,32.91,...,55.92,36.49,34.96,23.50,26.14,36.11,33.69,38.25,47.01,36.50
2007-01-08,100.11,23.22,65.26,124.14,54.86,36.97,72.30,26.53,16.27,32.95,...,55.92,36.67,35.09,23.57,26.18,36.09,33.73,38.31,47.49,36.66


In [3]:
price_data_for_plot = []
for col in etf_prices.columns:
    trace = go.Scatter(
                x=etf_prices.index,
                y=etf_prices[col],
                name=col,
                opacity = 0.8)
    price_data_for_plot.append(trace)

layout = dict(
    title = "ETF Prices from 2007 to 2016",
    xaxis = dict(
        range = [etf_prices.index])
)

fig = dict(data=price_data_for_plot, layout=layout)
py.iplot(fig, filename = "ETF Prices from 2007 to 2016")

The draw time for this plot will be slow for all clients.


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/plotly/api/v1/clientresp.py:40: UserWarning:

Estimated Draw Time Too Long



# Compute log return

In [4]:
etf_returns_df = np.log(etf_prices) - np.log(etf_prices.shift(1))
etf_returns_df = etf_returns_df.dropna()
etf_returns_df.head()

,AGG,DBC,DFE,DIA,DXJ,EEM,EFA,EWG,EWH,EWI,...,XLE,XLF,XLI,XLK,XLP,XLU,XLV,XLY,XME,XOP
date,,,,,,,,,,,,,,,,,,,,,
2007-01-03,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2007-01-04,0.002100,-0.026396,-0.000914,0.001686,0.012047,-0.013920,-0.002588,-0.003320,-0.010576,-0.002397,...,-0.018372,-0.003256,-0.004267,0.015313,0.001520,-0.001359,0.011279,-0.000260,-0.005048,-0.013187
2007-01-05,-0.000699,0.000862,-0.008881,-0.007568,-0.021133,-0.029524,-0.015393,-0.016770,-0.007533,-0.012981,...,0.006639,-0.008460,-0.003427,-0.008053,-0.007242,-0.018112,-0.005624,-0.006774,-0.008684,0.009359
2007-01-08,0.000600,0.000862,0.003684,0.003227,0.001459,0.007058,0.001384,-0.003011,0.024892,0.001215,...,0.000000,0.004921,0.003712,0.002974,0.001529,-0.000554,0.001187,0.001567,0.010159,0.004374
2007-01-09,0.000000,-0.008651,-0.000153,-0.000564,0.007084,-0.022430,0.000968,0.004513,-0.002462,-0.000607,...,-0.008621,-0.001638,-0.000570,0.001272,0.002289,0.000831,-0.001187,0.005467,-0.011436,-0.010420


We calculate the return using

$$ R = p^T w $$

where $R$ is the expected return, $p^T$ is the transpose of the vector for the mean
returns for each time series and w is the weight vector of the portfolio. $p$ is a $N \times 1$
column vector, so $p^T$ turns is a $1 \times N$ row vector which can be multiplied with the
$N \times 1$ weight (column) vector w to give a scalar result. This is equivalent to the dot
product used in the code. Keep in mind that `Python` has a reversed definition of
rows and columns and the accurate `NumPy` version of the previous equation would
be `R = w * p.T`

Next, we calculate the standard deviation

$$\sigma = \sqrt{w^T C w}$$

where $C$ is the $N \times N$ covariance matrix of the returns. Please
note that if we simply calculated the simple standard deviation with the appropriate weighting using `std(array(ret_vec).T*w)` we would get a slightly different
’bullet’. This is because the simple standard deviation calculation would not take
covariances into account. In the covariance matrix, the values on the diagonal
represent the simple variances of each asset, while the off-diagonal entries are the variances between the assets. By using ordinary `std()` we effectively only regard the
diagonal and miss the rest. A small but significant difference.

# Portfolio Optimization

Assumptions:
1. Uniform annual expense ratio of 0.75%.
2. Frequency of portfolio rebalance: once a month (25 trading days).
3. Expected annual return: 10%.

https://www.investopedia.com/ask/answers/071816/how-are-etf-fees-deducted.asp

In [5]:
def get_covariance_returns(returns):
    """
    Calculate covariance matrices.

    Parameters
    ----------
    returns : DataFrame
        Returns for each ticker and date

    Returns
    -------
    returns_covariance  : 2 dimensional Ndarray
        The covariance of the returns
    """
    
    return np.cov(returns.fillna(0).T)

In [6]:
def adjust_weights(weights):
    adjusted_weights = weights.copy()
    assets = list(weights.keys())
    fracs = list(weights.values())
    fracs = fracs[:-1]
    fracs = [round(frac, 4) for frac in fracs]

    min_weight_asset_id = min(enumerate(fracs), key=itemgetter(1))[0]
    min_weight_asset = assets[min_weight_asset_id]
    adjusted_weights[min_weight_asset] = 1 - sum(fracs) + adjusted_weights[min_weight_asset]
    return adjusted_weights

In [7]:
def get_optimal_weights(covariance_returns, return_vec, expected_annual_return):
    """
    Find the optimal weights.

    Parameters
    ----------
    covariance_returns : 2 dimensional Ndarray
        The covariance of the returns
    return_vec : 2 dimensional Ndarray
        Index weights for all tickers at a period in time
    expected_annual_return : float
        Expected rate of annual portfolio return
        
    Returns
    -------
    reduced_weights : 1 dimensional Ndarray
        The solution for x
    etfs_selection: 1 dimensional Ndarray
    
    """
    M = 100
    num_assets = covariance_returns.shape[1]
    initial_weights = cvx.Variable(num_assets)
    reduced_weights = cvx.Variable(num_assets)
    etfs_selection = cvx.Variable(num_assets, boolean=True)
    portfolio_variance = cvx.quad_form(reduced_weights, covariance_returns)
    
    ## Objective function
    objective = cvx.Minimize(portfolio_variance)
    
    ## Constraints
    mu = return_vec * reduced_weights
    rate_of_return = np.log(1 + expected_annual_return) * len(return_vec) / 252
    constraints = [initial_weights >= 0.001, initial_weights <= 0.3, sum(reduced_weights)==1,
                   reduced_weights - initial_weights >= -M * (1 - etfs_selection),
                   reduced_weights - initial_weights <= M * (1 - etfs_selection),
                   reduced_weights >= -M * etfs_selection, reduced_weights <= M * etfs_selection,
                   sum(etfs_selection) >= 5, sum(etfs_selection) <= 10, sum(mu) >= rate_of_return]
    
    ## Construct and solve the problem
    prob = cvx.Problem(objective, constraints)
    prob.solve()
    
    return reduced_weights.value, etfs_selection.value

# Rebalance Portfolio Over Time

Adopt a calendar-based rebalancing every 3 months

In [8]:
def rebalance_portfolio(returns, expected_return, expense_ratio,
                        shift_size, chunk_size):
    """
    Get weights for each rebalancing of the portfolio.

    Parameters
    ----------
    returns : DataFrame
        Returns for each ticker and date
    expected_return : float
        Expected rate of return for the portfolio
    shift_size : int
        The number of days between each rebalance
    chunk_size : int
        The number of days to look in the past for rebalancing

    Returns
    -------
    all_rebalance_weights  : list of Ndarrays
        The ETF weights for each point they are rebalanced
    """
    #yearly_trading_days = 252
    all_rebalance_weights = []
    dates_with_rebalance = []
    assets_tickers = list(returns.columns)
    num_assets = len(assets_tickers)
    for end in np.arange(shift_size, len(returns), shift_size):
        rate_of_return = expected_return
        if end - chunk_size < 0:
            continue
        start = end - chunk_size
        partial_returns = returns[start:end]
        #dates_with_rebalance.append(returns.index[end])
        covariance_returns = get_covariance_returns(partial_returns)
        rebalanced_asset_weights = None
        while rebalanced_asset_weights is None:
            rebalanced_asset_weights, assets_selection = get_optimal_weights(covariance_returns,
                                                                             partial_returns.values,
                                                                             rate_of_return + expense_ratio)
            rate_of_return -= 0.01
        rebalanced_asset_weights = [round(weight, 4) for weight in rebalanced_asset_weights]
        selected_asset_weights = {}
        for i in range(num_assets):
            if abs(assets_selection[i] - 1) < 0.001 and rebalanced_asset_weights[i] > 0.01:
                selected_asset_weights[assets_tickers[i]] = rebalanced_asset_weights[i]
        print("Rebalanced portfolios on %s:" % returns.index[end])
        print(selected_asset_weights)
        print("Actual achievable return: %s" % round(rate_of_return + 0.01, 4))
        print("===========================================")
        print()
        selected_asset_weights['date'] = returns.index[end]
        all_rebalance_weights.append(selected_asset_weights)

    return all_rebalance_weights

In [18]:
# Acquire ETF price data
etf_prices = pd.read_csv('data/prices.txt', sep=',')
etf_prices = etf_prices.rename(columns={etf_prices.columns[0]: "date"}).set_index('date')

# Remove US dollar
etf_prices = etf_prices.iloc[:, :-1]
print("Number of assets: %s" %etf_prices.shape[1])
print("Number of samples: %s" %etf_prices.shape[0])

# Calculate log return
etf_returns_df = np.log(etf_prices) - np.log(etf_prices.shift(1))
etf_returns_df = etf_returns_df.dropna()

chunk_size = 252
shift_size = 126
expected_return = 0.1
annual_expense_ratio = 0.0075
print("Rebalancing portfolios...")
start_time = time.time()
all_rebalanced_weights = rebalance_portfolio(etf_returns_df, expected_return,
                                             annual_expense_ratio, shift_size, chunk_size)
print("Elapsed time: %s seconds" %(round(time.time() - start_time, 4)))
adjusted_rebalanced_weights = []

# Reduce ETFs with negligible weights
for weight in all_rebalanced_weights:
    adjusted_rebalanced_weights.append(adjust_weights(weight))
    
with open('portfolio.json', 'w') as f:
    json.dump(adjusted_rebalanced_weights, f, indent=4)

Number of assets: 52
Number of samples: 2521
Rebalancing portfolios...
Rebalanced portfolios on 2008-01-02:
{'AGG': 0.225, 'EWZ': 0.1118, 'IAU': 0.0672, 'SDS': 0.0612, 'SH': 0.3, 'SPY': 0.0981, 'USO': 0.0355, 'XLU': 0.0993}
Actual achievable return: 0.1

Rebalanced portfolios on 2008-07-02:
{'AGG': 0.3, 'IAU': 0.081, 'IVV': 0.1819, 'SH': 0.3, 'XOP': 0.133}
Actual achievable return: 0.1

Rebalanced portfolios on 2008-12-29:
{'AGG': 0.3, 'GLD': 0.2198, 'IBB': 0.0678, 'SDS': 0.1074, 'SH': 0.3}
Actual achievable return: 0.1

Rebalanced portfolios on 2009-06-25:
{'AGG': 0.2993, 'GLD': 0.2999, 'IBB': 0.1337, 'LQD': 0.2999}
Actual achievable return: 0.0

Rebalanced portfolios on 2009-12-21:
{'AGG': 0.3, 'EWT': 0.0558, 'EWZ': 0.1569, 'SH': 0.3, 'SLV': 0.035, 'XLV': 0.14}
Actual achievable return: 0.1

Rebalanced portfolios on 2010-06-15:
{'AGG': 0.3, 'GLD': 0.1535, 'IYR': 0.1454, 'LQD': 0.2483, 'SH': 0.1508}
Actual achievable return: 0.1

Rebalanced portfolios on 2010-12-09:
{'AGG': 0.3, 'EWH'

In [19]:
adjusted_rebalanced_weights_df = pd.DataFrame(data=adjusted_rebalanced_weights)
adjusted_rebalanced_weights_df = adjusted_rebalanced_weights_df.fillna(0)
adjusted_rebalanced_weights_df.head()

,AGG,DFE,DXJ,EWH,EWT,EWZ,GLD,IAU,IBB,ITB,...,SLV,SPY,USO,XHB,XLF,XLP,XLU,XLV,XOP,date
0,0.2250,0.0,0.0,0.0,0.0000,0.1118,0.0000,0.0672,0.0000,0.0,...,0.0000,0.0981,0.0374,0.0,0.0,0.0,0.0993,0.00,0.000,2008-01-02
1,0.3000,0.0,0.0,0.0,0.0000,0.0000,0.0000,0.0851,0.0000,0.0,...,0.0000,0.0000,0.0000,0.0,0.0,0.0,0.0000,0.00,0.133,2008-07-02
2,0.3000,0.0,0.0,0.0,0.0000,0.0000,0.2198,0.0000,0.0728,0.0,...,0.0000,0.0000,0.0000,0.0,0.0,0.0,0.0000,0.00,0.000,2008-12-29
3,0.2993,0.0,0.0,0.0,0.0000,0.0000,0.2999,0.0000,0.1009,0.0,...,0.0000,0.0000,0.0000,0.0,0.0,0.0,0.0000,0.00,0.000,2009-06-25
4,0.3000,0.0,0.0,0.0,0.0558,0.1569,0.0000,0.0000,0.0000,0.0,...,0.0473,0.0000,0.0000,0.0,0.0,0.0,0.0000,0.14,0.000,2009-12-21


# Calculate cumulative returns over time

In [20]:
holding_shares_df = pd.DataFrame(columns=adjusted_rebalanced_weights_df.columns)
holding_shares_df['date'] = adjusted_rebalanced_weights_df['date']
holding_shares_df['balance'] = 0
holding_shares_df = holding_shares_df.fillna(0)

### Calculate number of holding shares for each asset
Assume constant amount of $10,000 maintained in the portfolio across the period of time. The number of shares of each asset can be calculated.

In [21]:
dates_with_rebalance = adjusted_rebalanced_weights_df['date']
cols = list(adjusted_rebalanced_weights_df.columns)
cols.remove('date')
partial_etf_prices = etf_prices.loc[dates_with_rebalance, cols].reset_index()
partial_etf_prices.head()

,date,AGG,DFE,DXJ,EWH,EWT,EWZ,GLD,IAU,IBB,...,SH,SLV,SPY,USO,XHB,XLF,XLP,XLU,XLV,XOP
0,2008-01-02,101.65,59.88,50.60,21.79,14.49,80.95,84.86,8.49,80.77,...,61.50,15.18,144.93,78.47,18.69,28.36,28.39,42.09,34.94,52.90
1,2008-07-02,99.55,48.22,48.20,16.27,13.58,83.96,93.17,9.34,78.25,...,69.50,18.22,126.14,116.84,15.90,20.03,26.66,40.80,30.37,68.84
2,2008-12-29,104.09,25.06,38.04,10.19,7.19,32.70,86.35,8.64,68.13,...,75.11,10.77,86.91,30.92,11.34,11.68,23.27,28.14,25.81,28.37
3,2009-06-25,101.92,28.33,38.08,13.83,10.14,53.22,92.31,9.24,72.15,...,65.64,13.81,92.08,38.05,11.91,11.86,23.11,27.84,26.37,31.60
4,2009-12-21,104.31,35.89,39.50,15.54,12.46,73.37,106.95,10.70,80.32,...,52.63,16.69,111.33,36.16,15.09,14.43,26.38,31.44,31.42,41.20


In [26]:
holding_shares_df = pd.DataFrame(columns=adjusted_rebalanced_weights_df.columns)
holding_shares_df['date'] = adjusted_rebalanced_weights_df['date']
holding_shares_df['balance'] = 0
holding_shares_df = holding_shares_df.fillna(0)
etfs = list(adjusted_rebalanced_weights_df.columns)
etfs.remove('date')
initial_balance = 10000
rebalance_count = 0
while rebalance_count < holding_shares_df.shape[0]:
    if rebalance_count == 0:
        balance = initial_balance
    else:
        balance = 0
        for etf in etfs:
            balance += holding_shares_df.loc[rebalance_count-1, etf] * partial_etf_prices.loc[rebalance_count, etf]
    holding_shares_df.loc[rebalance_count, 'balance'] = round(balance, 2)
    for etf in etfs:
        holding_shares_df.loc[rebalance_count, etf] = balance * adjusted_rebalanced_weights_df.loc[rebalance_count, etf] / partial_etf_prices.loc[rebalance_count, etf]
        holding_shares_df.loc[rebalance_count, etf] = round(holding_shares_df.loc[rebalance_count, etf], 2)
    rebalance_count += 1

holding_shares_df = holding_shares_df.set_index('date')
holding_shares_df.head()

,AGG,DFE,DXJ,EWH,EWT,EWZ,GLD,IAU,IBB,ITB,...,SLV,SPY,USO,XHB,XLF,XLP,XLU,XLV,XOP,balance
date,,,,,,,,,,,,,,,,,,,,,
2008-01-02,22.13,0.0,0.0,0.0,0.00,13.81,0.00,79.15,0.00,0.0,...,0.00,6.77,4.77,0.0,0.0,0.0,23.59,0.00,0.00,10000.00
2008-07-02,32.04,0.0,0.0,0.0,0.00,0.00,0.00,96.86,0.00,0.0,...,0.00,0.00,0.00,0.0,0.0,0.0,0.00,0.00,20.54,10630.82
2008-12-29,27.49,0.0,0.0,0.0,0.00,0.00,24.28,0.00,10.19,0.0,...,0.00,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.00,9538.10
2009-06-25,26.49,0.0,0.0,0.0,0.00,0.00,29.31,0.00,12.62,0.0,...,0.00,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.00,9022.26
2009-12-21,28.03,0.0,0.0,0.0,43.65,20.84,0.00,0.00,0.00,0.0,...,27.62,0.00,0.00,0.0,0.0,0.0,0.00,43.43,0.00,9747.17


# Generate random portfolios

10 random portfolios with 5-10 ETFs are simulated.

In [23]:
def generate_rand_weights(num_assets):
    seeds = []
    for i in range(num_assets):
        seeds.append(random.randint(1,10))
    
    return [round(seed / sum(seeds), 4) for seed in seeds]

In [24]:
N = 10 # No of random portfolios
portfolios = pd.DataFrame()
portfolios['Optimized Portfolio'] = holding_shares_df['balance']

for portfolio_id in range(N):
    # Generate no of assets in the portfolio
    n_etfs = random.randint(5,10)
    etf_pools = list(etf_prices.columns)
    rand_etfs = dict(zip(random.sample(etf_pools, n_etfs), 
                         generate_rand_weights(n_etfs)))
    
    # Initialize random portfolio
    porfolio_name = 'Random Portfolio ' + str(portfolio_id+1)
    portfolios[porfolio_name] = 0
    initial_balance = 10000
    holding_shares = {}
    start_point = '2008-01-02'
    for etf in rand_etfs:
        holding_shares[etf] = initial_balance * rand_etfs.get(etf) / etf_prices.loc[start_point, etf]

    for compare_point in portfolios.index:
        if compare_point == start_point:
            balance = initial_balance
        else:
            balance = 0
            for etf in rand_etfs:
                balance += holding_shares.get(etf) * etf_prices.loc[compare_point, etf]
                
        balance = round(balance, 2)
        portfolios.loc[compare_point, porfolio_name] = balance

portfolios.head()

,Optimized Portfolio,Random Portfolio 1,Random Portfolio 2,Random Portfolio 3,Random Portfolio 4,Random Portfolio 5,Random Portfolio 6,Random Portfolio 7,Random Portfolio 8,Random Portfolio 9,Random Portfolio 10
date,,,,,,,,,,,
2008-01-02,10000.00,10000.00,10000.00,10000.00,10000.00,10000.00,10000.00,10000.00,10000.00,10000.00,10000.00
2008-07-02,10630.82,9541.21,8521.43,8763.34,10953.87,9031.50,8958.59,10670.23,9102.19,9854.44,10202.28
2008-12-29,9538.10,6547.97,5929.02,5431.02,5501.75,5369.66,6150.64,7218.33,6998.47,7986.72,5394.62
2009-06-25,9022.26,7440.67,6370.35,5802.36,6073.66,5596.65,6896.66,7544.86,6934.65,8033.15,6248.58
2009-12-21,9747.17,8769.73,7912.47,7310.85,6910.40,6746.25,7975.15,8020.58,7665.09,8685.00,7591.20


### Plot portfolio holding value over time

In [25]:
portfolio_holdings = []
# portfolios['Optimized Portfolio'] = optimal_portfolio_holding_shares_df['balance']
for portfolio in portfolios.columns:
    if portfolio == 'Optimized Portfolio':
        trace = go.Scatter(
            x = portfolios.index,
            y = portfolios[portfolio],
            name = portfolio,
            line = dict(color = '#FC4A1A'),
            opacity = 0.8
        )
    else:
        trace = go.Scatter(
            x = portfolios.index,
            y = portfolios[portfolio],
            name = portfolio,
            line = dict(color = '#7F7F7F'),
            opacity = 0.8
        )

    portfolio_holdings.append(trace)

layout = dict(
    title = "Portfolio Holding Values from 2008 to 2016",
    xaxis = dict(
        range = [portfolios.index])
)

fig = dict(data=portfolio_holdings, layout=layout)
py.iplot(fig, filename = "Portfolio Holding Values from 2008 to 2016")